In [1]:
import numpy as np
import pandas as pd
from math import sqrt, exp, log
from scipy.stats import norm
import os

In [62]:
def main():
    global S, T, R, K, Price
    ave, std, Date = [], [], []
    year = "2021"
    file = "加權指數.xlsx"
    path = "/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/"
    DD = pd.read_excel(path + file, sheet_name = year)

    DD_len = len(DD)
    
    for i in range(DD_len):
        D = pd.read_csv(DD['Path'].iloc[0] + '/' + DD['File'].iloc[i], low_memory = False)
        
        S = DD['S0'].iloc[i]
        T = DD['Maturity'].iloc[i]/365
        R = 0.01465
    
        Contract = 'TXO'
        Call_Put = 'C'
        Volume = 30
        m = 0
        IV = []

        D['Maturity_contract'] = D['Maturity_contract'].astype(str).str.strip()
        DD['Contract'] = DD['Contract'].astype(str).str.replace('-', '', regex = False).str.strip()
        
        for j in range(len(D)):
            if (
                (D['ID'].iloc[j] == Contract) and
                (D['Call_Put'].iloc[j] == Call_Put) and
                (str(D['Maturity_contract'].iloc[j])[:6] == DD['Contract'].iloc[i]) and
                (D['Num'].iloc[j] >= Volume)
            ):
                K = D["E"].iloc[j]
                Price = D["Pr"].iloc[j]               
                sol = solve(D['Call_Put'].iloc[j])
                if sol < 0.03:
                    continue
                IV.append(sol)
        
                m += 1
        
        ave.append(np.average(IV))
        std.append(np.std(IV, ddof = 1))
        Date.append(DD['Date'].iloc[i])
        print("%s \t ave = %.4f%% \t std.s = %f" % (str(Date[i]), ave[i] * 100, std[i]))
        print('**************************************************************')
        
    ave_series = pd.Series(ave).interpolate()
    std_series = pd.Series(std).interpolate()

    fp = open("Finalresult_" + year + "_" + Call_Put + ".txt", "w")
    for i in range(DD_len):
        print("%s \t %f \t %f" % (str(Date[i]), ave_series[i], std_series[i]), file = fp)
    fp.close()

def BS(X, Sigma):
    d1 = (log(S / K) + (R + Sigma * Sigma / 2) * T) / (Sigma * sqrt(T))
    d2 = d1 - Sigma * sqrt(T)
   
    if X == "C":
        return S * norm.cdf(d1) - K * exp(-R * T) * norm.cdf(d2) - Price
    if X == "P":
        return K * exp(-R * T) * norm.cdf(-d2) - S * norm.cdf(-d1) - Price

def solve(X):
    a = 0.00001
    b = 1.00001
    for i in range(4096):
        c = (b + a) / 2
        if b - c < pow(10, -6):
            break
        else:
            if BS(X, b) * BS(X, c) < 0:
                a = c
            else:
                b = c
    return c

In [63]:
main()

2021-01-04 00:00:00 	 ave = 7.8278% 	 std.s = 0.043359
**************************************************************
2021-01-05 00:00:00 	 ave = 6.3891% 	 std.s = 0.042968
**************************************************************
2021-01-06 00:00:00 	 ave = 8.3782% 	 std.s = 0.051416
**************************************************************
2021-01-07 00:00:00 	 ave = 10.8930% 	 std.s = 0.036717
**************************************************************
2021-01-08 00:00:00 	 ave = 9.8984% 	 std.s = 0.033314
**************************************************************
2021-01-11 00:00:00 	 ave = 10.4000% 	 std.s = 0.038635
**************************************************************
2021-01-12 00:00:00 	 ave = 10.9397% 	 std.s = 0.040237
**************************************************************
2021-01-13 00:00:00 	 ave = 14.9876% 	 std.s = 0.059480
**************************************************************
2021-01-14 00:00:00 	 ave = 21.2716% 	 std.s = 0.021

/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2021-03-09 00:00:00 	 ave = 17.6811% 	 std.s = nan
**************************************************************
2021-03-10 00:00:00 	 ave = 9.7292% 	 std.s = 0.064701
**************************************************************
2021-03-11 00:00:00 	 ave = 16.2111% 	 std.s = 0.028274
**************************************************************
2021-03-12 00:00:00 	 ave = 15.8147% 	 std.s = 0.017884
**************************************************************
2021-03-15 00:00:00 	 ave = 18.1060% 	 std.s = 0.028213
**************************************************************
2021-03-16 00:00:00 	 ave = 17.1270% 	 std.s = 0.054754
**************************************************************
2021-03-17 00:00:00 	 ave = 4.7298% 	 std.s = 0.013724
**************************************************************
2021-03-18 00:00:00 	 ave = 14.2703% 	 std.s = 0.010359
**************************************************************
2021-03-19 00:00:00 	 ave = 15.7040% 	 std.s = 0.025542

/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2021-03-22 00:00:00 	 ave = nan% 	 std.s = nan
**************************************************************


...

2021-12-17 00:00:00 	 ave = nan% 	 std.s = nan
**************************************************************
2021-12-20 00:00:00 	 ave = 13.3255% 	 std.s = 0.013546
**************************************************************
2021-12-21 00:00:00 	 ave = 11.3234% 	 std.s = 0.009031
**************************************************************
2021-12-22 00:00:00 	 ave = 12.2774% 	 std.s = 0.014305
**************************************************************
2021-12-23 00:00:00 	 ave = 11.7663% 	 std.s = 0.007292
**************************************************************
2021-12-24 00:00:00 	 ave = 12.2899% 	 std.s = 0.003008
**************************************************************
2021-12-27 00:00:00 	 ave = 12.9254% 	 std.s = 0.015973
**************************************************************


/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2021-12-28 00:00:00 	 ave = nan% 	 std.s = nan
**************************************************************


/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/xinc./Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2021-12-29 00:00:00 	 ave = nan% 	 std.s = nan
**************************************************************
2021-12-30 00:00:00 	 ave = 7.3161% 	 std.s = 0.025319
**************************************************************
